## Test Data Prediction
### Feat7
This notebook will be used to open test.csv file and use designated model to predict the test data for kaggle submission.

In [15]:
%%bash
git branch

  CV_check
  EDA1/wip
  cost1/feat1/wip
  cost1/feat2/wip
  cost1/wip
  feat1/score1/wip
  feat1/wip
  feat2/score1/wip
  feat2/wip
  feat3/wip
  feat4/wip
  feat5/wip
  feat6/wip
* feat7(EDA1)/wip
  master


In [3]:
%%bash
git add 'Test_data.ipynb'
git add 'Titanic.ipynb'
git commit -m 'Prepare code for predicting for submission file'
git push --set-upstream origin master

[feat7(EDA1)/wip 4c24f24] Prepare code for predicting for submission file
 1 file changed, 528 insertions(+), 751 deletions(-)
 rewrite Test_data.ipynb (85%)
Branch master set up to track remote branch master from origin.


Everything up-to-date


In [28]:
model_ver = 'feat7.sav'

In [29]:
%matplotlib inline 
import requests 
import StringIO
import zipfile
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 

# If this module is not already installed, you may need to install it. 
# You can do this by typing 'pip install seaborn' in the command line
import seaborn as sns 

import sklearn.model_selection
import pickle

Load data from csv file then prepare the data before inputting into the trained model.

In [30]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [31]:
X = train.copy()
#X = X.drop('Survived',axis=1)
X_plot = train.copy()
X = X.drop('PassengerId',axis=1)  # Shouldn't relate to survival chance
X = X.drop('Cabin',axis=1)
X = X.drop('Ticket',axis=1)

#Mirror action on X to test
#test = test.drop('PassengerID',axis=1)
test = test.drop('PassengerId',axis=1)
test = test.drop('Cabin',axis=1)
test = test.drop('Ticket',axis=1)

#X = X.drop('Name',axis=1)
Y = pd.DataFrame(train['Survived'])

X_plot['Initial']=0
for i in X_plot:
    X_plot['Initial']=X_plot.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations
#X_plot['Initial'].value_counts()

#Change data in X also (since its the real training data)
X['Initial']=0
for i in X:
    X['Initial']=X.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations
#Also perform same actions on test dataframe
test['Initial']=0
for i in X:
    test['Initial']=test.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations
X['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)
test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)

X['Age'][(X['Initial']=='Mr') & (X['Age'].isnull())] = X[X['Initial']=='Mr']['Age'].mean()
X['Age'][(X['Initial']=='Miss') & (X['Age'].isnull())] = X[X['Initial']=='Miss']['Age'].mean()
X['Age'][(X['Initial']=='Mrs') & (X['Age'].isnull())] = X[X['Initial']=='Mrs']['Age'].mean()
X['Age'][(X['Initial']=='Master') & (X['Age'].isnull())] = X[X['Initial']=='Master']['Age'].mean()
X['Age'][(X['Initial']=='other') & (X['Age'].isnull())] = X[X['Initial']=='other']['Age'].mean()

test['Age'][(test['Initial']=='Mr') & (test['Age'].isnull())] = test[test['Initial']=='Mr']['Age'].mean()
test['Age'][(test['Initial']=='Miss') & (test['Age'].isnull())] = test[test['Initial']=='Miss']['Age'].mean()
test['Age'][(test['Initial']=='Mrs') & (test['Age'].isnull())] = test[test['Initial']=='Mrs']['Age'].mean()
test['Age'][(test['Initial']=='Master') & (test['Age'].isnull())] = test[test['Initial']=='Master']['Age'].mean()
test['Age'][(test['Initial']=='other') & (test['Age'].isnull())] = test[test['Initial']=='other']['Age'].mean()

num_attri = ['Age','SibSp','Parch','Fare']
cat_attri = [column for column in X if (column not in num_attri and column != 'Survived')]

for column in (cat_attri):
    #Lets just keep every categorical attributes for plotting
    tempDF = pd.get_dummies(X[column],prefix=column)
    X.drop(column,axis=1,inplace=True)
    X = X.join(tempDF)
for column in (cat_attri): #Create dummy for test
    #Lets just keep every categorical attributes for plotting
    tempDF = pd.get_dummies(test[column],prefix=column)
    test.drop(column,axis=1,inplace=True)
    test = test.join(tempDF)

#Concat X(train data) with X from test data to avoid mismatch feature
train_objs_num = len(X)
dataset = pd.concat(objs=[X, test], axis=0)
#dataset = pd.get_dummies(dataset)
X = dataset[:train_objs_num].copy()
test = dataset[train_objs_num:].copy()

/Users/theerit/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel/__main__.py:19: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/Users/theerit/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel/__main__.py:25: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/Users/theerit/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel/__main__.py:29: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/Users/theerit/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set o

In [33]:
loaded_model = pickle.load(open(model_ver, 'rb'))
Y_predict = loaded_model.predict(test)
Y_predict

ValueError: feature_names mismatch: [u'Age', u'Embarked_C', u'Embarked_Q', u'Embarked_S', u'Fare', u'Parch', u'Pclass_1', u'Pclass_2', u'Pclass_3', u'Sex_female', u'Sex_male', u'SibSp'] [u'Age', u'Embarked_C', u'Embarked_Q', u'Embarked_S', u'Fare', u'Initial_Dona', u'Initial_Master', u'Initial_Miss', u'Initial_Mr', u'Initial_Mrs', u'Initial_Other', u'Name_Abbing, Mr. Anthony', u'Name_Abbott, Master. Eugene Joseph', u'Name_Abbott, Mr. Rossmore Edward', u'Name_Abbott, Mrs. Stanton (Rosa Hunt)', u'Name_Abelseth, Miss. Karen Marie', u'Name_Abelseth, Mr. Olaus Jorgensen', u'Name_Abelson, Mr. Samuel', u'Name_Abelson, Mrs. Samuel (Hannah Wizosky)', u'Name_Abrahamsson, Mr. Abraham August Johannes', u'Name_Abrahim, Mrs. Joseph (Sophie Halaut Easu)', u'Name_Adahl, Mr. Mauritz Nils Martin', u'Name_Adams, Mr. John', u'Name_Ahlin, Mrs. Johan (Johanna Persdotter Larsson)', u'Name_Aks, Master. Philip Frank', u'Name_Aks, Mrs. Sam (Leah Rosen)', u'Name_Albimona, Mr. Nassef Cassem', u'Name_Aldworth, Mr.

In [ ]:
test_temp = pd.read_csv('Test.csv')
sub_file = pd.concat([test_temp['PassengerId'],pd.DataFrame(Y_predict)],axis=1)
sub_file.columns =['PassengerId','Survived']
sub_file = sub_file.astype(int)
#pd.options.display.float_format = '{:,.0f}'.format

In [ ]:
sub_file.head()
sub_file.to_csv(path_or_buf = submission_name,index=False)